In [ ]:
import torch
import torchvision.transforms.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_mnist_digits(batches, ki_kf, reconstr, output=True):
    batches_to_plot = 5

    batch_size = len(batches[0])
    print('Batch size:', batch_size)

    num_digits = len(batches[0]) * batches_to_plot
    print('Num digits: ', num_digits)
    
    batches = batches[:batches_to_plot]
    # Create a grid of subplots with 20 rows and 20 columns
    fig, axes = plt.subplots(nrows=5, ncols=16, figsize=(20, 20))
    # fig, axes = plt.subplots(num_digits, 20, figsize=(20, 20))

    # Flatten the axes array to access the individual subplots easily
    axes = axes.flatten()

    # batches = [batches[i:i+784] for i in range(0, len(batch_correl_list), 784)]

    for batch_number, batch in enumerate(batches):
        for i, digit in enumerate(batch):
            # Convert Torch tensor to a NumPy array and reshape it to (28, 28)
            digit_np = digit.view(28, 28).detach().numpy()

            # Plot the digit using matplotlib
            axes[batch_number*batch_size + i].imshow(digit_np, cmap='gray')
            axes[batch_number*batch_size + i].axis('off')

    if output:
        plt.savefig(f'/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/nn_outputs/MNIST/reconstruction_flavour=vanilla_kikf={ki_kf}_bs=16_reconstr={reconstr}.pdf'.lower())
    # # Remove any remaining empty subplots
    # for i in range(num_digits, 39):
    #     axes[i].axis('off')

    # plt.savefig(file)
    plt.show()
    

In [ ]:
import matplotlib.pyplot as plt

# Define a function to calculate cross-entropy divergence
def mse_divergence(input_batch, output_batch):
    # Flatten the input and output batches
    input_flat = input_batch.view(-1)
    output_flat = output_batch.view(-1)
    
    # Compute the cross-entropy loss
    loss = F.mse_loss(output_flat, input_flat, reduction='mean')
    
    return loss

In [ ]:
def moving_average(a, n=10):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

## 1, 2 and 3pt functions

In [ ]:
orig_images_3pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0.017/batch_avg_approx_all_1pt_1.0-2pt_0.017-3pt_simpleout')
reconstructions_3pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0.017/mixed_reconstr_avg_approx_all_1pt_1.0-2pt_0.017-3pt_simpleout')

In [ ]:
plot_mnist_digits(orig_images_3pt[500:], ki_kf='123', reconstr=False)

In [ ]:
new_reconstructions_3pt = []
for batch in reconstructions_3pt:
    tidied_batch = [batch[i:i+784] for i in range(0, len(batch), 784)]
    new_reconstructions_3pt.append(tidied_batch)
# reconstructions_2pt = new_reconstructions_2pt

In [ ]:
plot_mnist_digits(new_reconstructions_3pt[2200:], ki_kf='123', reconstr=True)

In [ ]:
# Test Gaussian blur
def gaussian_blur_mnist_digit(flattened_tensor, kernel_size=5, sigma=1.0):
    # Reshape the flattened tensor back to its original shape (28x28)
    image = flattened_tensor.view(28, 28).unsqueeze(0).float()

    # Apply Gaussian blur
    blurred_image = F.gaussian_blur(image, kernel_size, sigma)

    # Remove the extra batch dimension
    blurred_image = blurred_image.squeeze(0)

    # Flatten the blurred image back to a tensor
    blurred_tensor = blurred_image.view(-1)

    return blurred_tensor

In [ ]:
def plot_flattened_mnist_digit(flattened_digit):
    # Reshape the flattened tensor to its original shape (28x28)
    image = flattened_digit.view(28, 28)

    # Plot the digit
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

### Losses (Training) - 1,2&3pt

In [ ]:
# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0.017/loss_hist_all_1pt_1.0-2pt_0.017-3pt_simpleout'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(loaded_tensor[:150])
plt.xlabel('Training pass')
plt.ylabel('MSE Loss')
# plt.title('MSE training loss for 1,2,3pt cumulant decoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_dec_training_loss_123-1_1_0.017.pdf')
plt.show()


# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0.017/enc_loss_hist'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(moving_average(loaded_tensor[:350], 50))
plt.xlabel('Training pass')
plt.ylabel('Moving average MSE loss')
# plt.title('MSE training loss for 1&2pt cumulant encoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_enc_training_loss_123-1_1_0.017.pdf')
plt.show()

### Losses (Validation)

In [ ]:
# Notice that the reconstructions are the same tensor repeated |B| times, so we can MSE loss this straight away
mse_321 = []
mse_loss = torch.nn.MSELoss()
for train_pass in range(2000, 2300):
    flattened_orig_batch = orig_images_3pt[train_pass].flatten()
    flattened_reconstr = torch.concatenate(new_reconstructions_3pt[train_pass], axis=0)
    mse_321.append(mse_loss(flattened_orig_batch, flattened_reconstr))
print(sum(mse_321)/len(mse_321))


 ## 1pt and 2pt functions

In [ ]:
orig_images_2pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0/batch_avg_approx_all_1pt_1.0-2pt_0.0-3pt_simpleout')
reconstructions_2pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0/mixed_reconstr_avg_approx_all_1pt_1.0-2pt_0.0-3pt_simpleout')
len(reconstructions_2pt)
# stiff_reconstructions = torch.load('/home/as/astapleton_phd/dsb/subtle_diffusion/correlators/correlator_nn/reconstr/stiff_reconstr_avg_approx')

In [ ]:
plot_mnist_digits(orig_images_2pt, ki_kf='12', reconstr=False)

In [ ]:
new_reconstructions_2pt = []
for batch in reconstructions_2pt:
    tidied_batch = [batch[i:i+784] for i in range(0, len(batch), 784)]
    new_reconstructions_2pt.append(tidied_batch)
# reconstructions_2pt = new_reconstructions_2pt

In [ ]:
plot_mnist_digits(new_reconstructions_2pt[2300:], ki_kf='12', reconstr=True)

### Losses (Training) - 1&2pt

In [ ]:
# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0/enc_loss_hist'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(moving_average(loaded_tensor[:180], 50))
plt.xlabel('Training pass')
plt.ylabel('Moving average MSE loss')
# plt.title('MSE training loss for 1&2pt cumulant encoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_enc_training_loss_123-1_1_0.pdf')
plt.show()

In [ ]:
# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-1_3pt-0/loss_hist_all_1pt_1.0-2pt_0.0-3pt_simpleout'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(moving_average(loaded_tensor[4:200], 1))
plt.xlabel('Training pass')
plt.ylabel('MSE Loss')
# plt.title('MSE training loss for 1&2pt cumulant decoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_dec_training_loss_123-1_1_0.0.pdf')
plt.show()


### Losses (Validation)

In [ ]:
# Notice that the reconstructions are the same tensor repeated |B| times, so we can MSE loss this straight away
mse_21 = []
mse_loss = torch.nn.MSELoss()
for train_pass in range(2000, 2300):
    flattened_orig_batch = orig_images_2pt[train_pass].flatten()
    flattened_reconstr = torch.concatenate(new_reconstructions_2pt[train_pass], axis=0)
    mse_21.append(mse_loss(flattened_orig_batch, flattened_reconstr))
print(sum(mse_21)/len(mse_21))

## 1pt only

In [ ]:
orig_images_1pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-0_3pt-0/batch_avg_approx_all_1pt_0.0-2pt_0.0-3pt_simpleout')
reconstructions_1pt = torch.load('/Users/as/Desktop/reconstructions/moments/MNIST/2pt-0_3pt-0/mixed_reconstr_avg_approx_all_1pt_0.0-2pt_0.0-3pt_simpleout')

In [ ]:
plot_mnist_digits(orig_images_1pt, ki_kf=1, reconstr=False)

In [ ]:
new_reconstructions_1pt = []
for batch in reconstructions_1pt:
    tidied_batch = [batch[i:i+784] for i in range(0, len(batch), 784)]
    new_reconstructions_1pt.append(tidied_batch)
# reconstructions_2pt = new_reconstructions_2pt

In [ ]:
plot_mnist_digits(new_reconstructions_1pt[900:], ki_kf='1', reconstr=True)

### Losses (Training) -- 1pt Only

In [ ]:
# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-0_3pt-0/enc_loss_hist'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(moving_average(loaded_tensor[:300], 50))
plt.xlabel('Training pass')
plt.ylabel('Moving average MSE loss')
# plt.title('MSE training loss for 1&2pt cumulant encoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_enc_training_loss_123-1_0_0.pdf')
plt.show()



# Step 1: Load the tensor from the saved file
file_path = '/Users/as/Desktop/reconstructions/moments/MNIST/2pt-0_3pt-0/loss_hist_all_1pt_0.0-2pt_0.0-3pt_simpleout'  # Replace with the path to your saved tensor file
loaded_tensor = torch.load(file_path)

# Step 2: Convert the tensor to a NumPy array
# numpy_array = loaded_tensor.numpy()

# Step 3: Plot the data using Matplotlib
plt.plot(loaded_tensor[0:150])
plt.xlabel('Training pass')
plt.ylabel('MSE Loss')
# plt.title('MSE training loss for 1 only cumulant decoder')
plt.grid(True)
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/MOMENTS_dec_training_loss_123-1_0_0.0.pdf')
plt.show()

### Losses (Validation)

In [ ]:
# Notice that the reconstructions are the same tensor repeated |B| times, so we can MSE loss this straight away
mse_1 = []
mse_loss = torch.nn.MSELoss()
for train_pass in range(2000, 2300):
    flattened_orig_batch = orig_images_1pt[train_pass].flatten()
    flattened_reconstr = torch.concatenate(new_reconstructions_1pt[train_pass], axis=0)
    mse_1.append(mse_loss(flattened_orig_batch, flattened_reconstr))
print(sum(mse_1)/len(mse_1))

### Combined MSE losses (val)

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
# Combined MSE validation
grainer = 1

plt.scatter(range(len(moving_average(mse_1, grainer))), moving_average(mse_1, grainer), label='1pt only')
plt.scatter(range(len(moving_average(mse_21, grainer))), moving_average(mse_21, grainer), label='1pt and 2pt')
plt.scatter(range(len(moving_average(mse_321, grainer))), moving_average(mse_321, grainer), label='1, 2 and 3pt')
plt.xlabel('Validation sample')
plt.ylabel('MSE Loss')
plt.legend()
plt.tight_layout()
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/loss_trifle_all.pdf')
plt.show()


plt.scatter(range(len(moving_average(mse_1, grainer))), moving_average(mse_1, grainer), label='1pt only')
plt.scatter(range(len(moving_average(mse_21, grainer))), moving_average(mse_21, grainer), label='1pt and 2pt')
plt.scatter(range(len(moving_average(mse_321, grainer))), moving_average(mse_321, grainer), label='1, 2 and 3pt')
plt.xlabel('Validation sample')
plt.ylabel('MSE Loss')
plt.legend()
plt.tight_layout()
plt.savefig('/Users/as/astapleton_phd/dsb/papers/cumulant_encoder/figs/since_iaifi/final/MNIST/loss_trifle_no_13.pdf')
plt.show()


plt.plot(moving_average(mse_1, 100), label='1pt only')
plt.plot(moving_average(mse_21, 100), label='1pt and 2pt')
plt.plot(moving_average(mse_321, 100), label='1, 2 and 3pt')
plt.legend()
plt.show()